

### Import



In [1]:
# Import all the necessary Library 
import torchvision
import torch.utils.data as utils
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import time
import copy
from torch.optim import lr_scheduler
import os
from PIL import Image
import torch
from torch.autograd import Variable
import PIL.ImageOps    
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pandas as pd 


In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.precision', 4)

### Functions and setups

In [3]:
def imshow(img,text=None,should_save=False):
    npimg = img.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()    

def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.show()

In [4]:
class SiameseNetworkDataset():
    
    def __init__(self,training_csv=None,training_dir=None,transform=None):
        # used to prepare the labels and images path
        self.training_df=pd.read_csv(training_csv)
        self.training_df.columns =["image1","image2","label"]
        self.training_dir = training_dir    
        self.transform = transform

    def __getitem__(self,index):
        
        # getting the image path
        image1_path=os.path.join(self.training_dir,self.training_df.iat[index,0])
        image2_path=os.path.join(self.training_dir,self.training_df.iat[index,1])
        
        
        # Loading the image
        img0 = Image.open(image1_path)
        img1 = Image.open(image2_path)
        img0 = img0.convert("L")
        img1 = img1.convert("L")
        
        # Apply image transformations
        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1 , torch.from_numpy(np.array([int(self.training_df.iat[index,2])],dtype=np.float32))
    
    def __len__(self):
        return len(self.training_df)

In [5]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        
        # Setting up the Sequential of CNN Layers
        self.cnn1 = nn.Sequential(
            
            nn.Conv2d(1, 96, kernel_size=11,stride=1),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(5,alpha=0.0001,beta=0.75,k=2),
            nn.MaxPool2d(3, stride=2),
            
            nn.Conv2d(96, 256, kernel_size=5,stride=1,padding=2),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(5,alpha=0.0001,beta=0.75,k=2),
            nn.MaxPool2d(3, stride=2),
            nn.Dropout2d(p=0.3),

            nn.Conv2d(256,384 , kernel_size=3,stride=1,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384,256 , kernel_size=3,stride=1,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, stride=2),
            nn.Dropout2d(p=0.3),

        )
        
        # Defining the fully connected layers
        self.fc1 = nn.Sequential(
            nn.Linear(30976, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.5),
            
            nn.Linear(1024, 128),
            nn.ReLU(inplace=True),
            
            nn.Linear(128,2))
        
  
  
    def forward_once(self, x):
        # Forward pass 
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        # forward pass of input 1
        output1 = self.forward_once(input1)
        # forward pass of input 2
        output2 = self.forward_once(input2)
        return output1, output2


In [6]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

### Evaluate Functions

In [7]:
def siamese_eval_dataload(test_image, datafolder):
    prediction_df = []
    for label in os.listdir(datafolder):
        for image in os.listdir(datafolder + label):
            prediction_df.append({
              'test_img':test_image,
              'img2': datafolder + label + "/" + image,
              'labels': label,   
            })

    prediction_df = pd.DataFrame(prediction_df)
    prediction_df.to_csv("../data/prediction.csv", index=False)

    # load prediction csv
    prediction_csv = "../data/prediction.csv"

    prediction_dataset = SiameseNetworkDataset(training_csv=prediction_csv,training_dir="",
                                          transform=transforms.Compose([transforms.Resize((105,105)),
                                                                        transforms.ToTensor()
                                                                        ]))

    prediction_dataloader = DataLoader(prediction_dataset, batch_size=1)

    return prediction_df, prediction_dataloader

In [8]:
def siamese_eval(test_image, datafolder, net, n=3):
    prediction_df, prediction_dataloader = siamese_eval_dataload(test_image, datafolder)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dissimilarity = []

    # Get dissimilarity for every combination of test img and our imgs
    for i, data in enumerate(prediction_dataloader,0): 
        test, compare, label = data
        concatenated = torch.cat((test, compare),0)
        output1, output2 = net(test.to(device), compare.to(device))
        eucledian_distance = F.pairwise_distance(output1, output2)
        dissimilarity.append(eucledian_distance.item()*100)

    dissim_df = pd.DataFrame({"score":dissimilarity,
                            "labels":prediction_df['labels']})
  
    # Get avg dissimilarity score for each label
    labels = []
    scores = []
    rmsquares = []
    for label in set(prediction_df['labels']):
        scores_for_label = dissim_df[dissim_df['labels']==label]['score'].tolist()
        labels.append(label)
        scores.append(sum(scores_for_label)/len(scores_for_label))
        rmsquare = np.sqrt(np.mean(np.square(scores_for_label)))
        rmsquares.append(rmsquare)

    # Get labels and the corresponding avg score
    avg_score_df = pd.DataFrame({"labels":labels,
                              "score":scores,
                              "root_mean_squared":rmsquares})
    
    top_n_pred = avg_score_df.sort_values("score", ascending=True)['labels'].head(n).tolist()
   
    return top_n_pred

### Eval

In [15]:
PATH = "./model_final.pt"
net = SiameseNetwork() 
net.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))
net.eval()
datafolder = "../data/manual-clusters/20220430/categorized_new/"
test_image = "../data/manual-clusters/20220430/categorized_new/1/gleeful-kangaroomouse-0.jpg"
results = siamese_eval(test_image, datafolder, net, n=3)
results

C:\Users\zongy\anaconda3\lib\site-packages\torch\nn\functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


['6', '1', '4']

In [26]:
testimgs = []
for label in os.listdir(datafolder):
    for image in os.listdir(datafolder + label):
        testimgs.append({
            'dir': datafolder + label + "/" + image,
            'true_label': label,   
        })
testimgs = pd.DataFrame(testimgs)

In [ ]:
# Get the predicated and true label
pred = []
true = []

from tqdm import tqdm
for i, row in tqdm(testimgs.iterrows()):
    test_image = row['dir']
    pred.append(siamese_eval(test_image, datafolder, net, n=3))
    true.append(row['true_label'])

In [ ]:
# Number of images with each labels
top_3_pred = pd.DataFrame({'pred':pred,
              'true':true})

top_3_pred['true'].value_counts()

In [ ]:
# Top 3 accuracy for each label
correct = []
for i, row in top_3_pred.iterrows():
    if row['true'] in row['pred']:
        correct.append(1)
    else:
        correct.append(0)
top_3_pred['correct'] = correct
print(top_3_pred.groupby(['true'])['correct'].mean())

# Top 1 accuracy for overall
top_3_pred['correct'].mean()